In [1]:
import pandas as pd
import parse_csm
import parse_imdb
from os import path
from tqdm import tqdm, tqdm_notebook
%load_ext autoreload
%autoreload 2

## Commonsense Media Parsing
Parse the collection of CSM html pages for requested information.  Write the output to an atomic, cleaned JSON file.

In [8]:
path_root = path.join('/Users/quinone/Downloads/portia_projects/hackathon/scrape-hand', 'movies')
path_file = path.join(path_root, 'movies', 'movie-reviews', 'zootopia.html')
path_data = path.join(path_root, 'data.json.gz')

list_files = parse_csm.discover_files(path_root)
# print(list_files)
list_results = []
for idx in tqdm_notebook(range(len(list_files))):
    parse_dict = parse_csm.parse_review_file(list_files[idx])
    if parse_dict is not None:
        list_results.append(parse_dict)
df_csm = pd.DataFrame(list_results)
list_results = None
print(df_csm)
df_csm.to_json(path_data, orient='records')


100%|██████████| 9802/9802 [30:43<00:00,  6.08it/s]   


                                   title  \
0                          4 Minute Mile   
1                      Alone in the Dark   
2                         Deck the Halls   
3     You Will Meet a Tall Dark Stranger   
4                                  Basic   
...                                  ...   
9797                           Zookeeper   
9798                      Burn Your Maps   
9799                           The Raven   
9800                             Aladdin   
9801                Diary of a Wimpy Kid   

                                                  brief  \
0     This film proves to be an interesting combinat...   
1                                                         
2     There's no doubt that Broderick and DeVito are...   
3     In many ways, &quot;You Will Meet a Tall Dark ...   
4     The "&quot;Basic&quot;" premise of this poorly...   
...                                                 ...   
9797  With so much comedic talent on board, it's rea...   

## IMDB extraction
Extract other information for IMDB linkage including financial facts and release date.

In [13]:
import re
path_imdb = path.join(path_root, '..', 'imdb-urls.new.txt')
df_imdb_read = pd.DataFrame(df[df['url_imdb'].str.contains('imdb')]['url_imdb'])
re_sub = re.compile(".*/([^/]+)/*?")
df_imdb_read['output'] = df_imdb_read['url_imdb'].apply(lambda x: re_sub.sub(r"\1", x.split("?")[0]).strip("/"))
df_imdb_read.sort_values('url_imdb').to_csv(path_imdb, sep=' ', index=False, header=False)


## IMDB Parsing
Parse the collection of IMDB html pages for requested information.  Write the output to an atomic, cleaned JSON file.

In [2]:
path_root = path.join('/Users/quinone/Downloads/portia_projects/hackathon/scrape-hand', 'imdb')
path_file = path.join(path_root, 'tt0092571.html')
path_data = path.join(path_root, '..', 'imdb.json.gz')

parse_dict = parse_imdb.parse_review_file(path_file)

list_files = parse_imdb.discover_files(path_root)
list_results = []
for idx in tqdm_notebook(range(len(list_files))):
    parse_dict = parse_imdb.parse_review_file(list_files[idx])
    if parse_dict is not None:
        list_results.append(parse_dict)
df_imdb = pd.DataFrame(list_results)
list_results = None
print(df_imdb)
df_imdb.to_json(path_data, orient='records')

/Users/quinone/anaconda/envs/cognita36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/quinone/anaconda/envs/cognita36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|██████████| 9625/9625 [18:32<00:00,  8.35it/s]


                         title                url  \
0                         Skin  /title/tt0964586/   
1                      Spymate  /title/tt0367088/   
2                       S1m0ne  /title/tt0258153/   
3     The Last of the Mohicans  /title/tt0104691/   
4                        Terri  /title/tt1687281/   
...                        ...                ...   
9420               Ghost World  /title/tt0162346/   
9421        The Akron Holy War  /title/tt7235502/   
9422                Ballet 422  /title/tt3164198/   
9423            The Last Mimzy  /title/tt0768212/   
9424           Black Hawk Down  /title/tt0265086/   

                                                 poster  \
0     https://m.media-amazon.com/images/M/MV5BMjA0Nz...   
1     https://m.media-amazon.com/images/M/MV5BMTU0Mj...   
2     https://m.media-amazon.com/images/M/MV5BNjZmNj...   
3     https://m.media-amazon.com/images/M/MV5BZDNiYm...   
4     https://m.media-amazon.com/images/M/MV5BMTkxOT...   
...      

In [77]:
        
# print (soup.prettify())

# s = Scraper()
# url1 = f'file:///{path_file}'
# data1 = {'title': 'Zootopia', # 'summary':'Charming buddy-animal story promotes tolerance, teamwork.',
#          'rating_mpaa':'PG', 'duration':'108 minutes', 'release':'2016',
#          'age_overall':'age 8+', #. 'age_parents':'age 9+', 'age_kids':'age 8+',
#          'imdb_url':'http://www.imdb.com/title/tt2948356',
#          'rating_kids':'rating-4', 'rating_adult':'rating-4', 'rating_overall':'rating-4',
#          'rating_adult_count':'156 reviews', 'rating_kids_count':'237 reviews',
#          #'summary':"""ZOOTOPIA is set in a world where walking, talking, "civilized" animals live in general harmony with one another, regardless of whether they're predator or prey. <span style="line-height: 20.8px;">When small-town rabbit Judy Hopps (voiced by <a href="https://www.commonsensemedia.org/search/Ginnifer%20Goodwin">Ginnifer Goodwin</a>) achieves her childhood dream of becoming the first rabbit to join the Zootopia Police Department, Chief of Police Bogo (<a href="https://www.commonsensemedia.org/search/Idris%20Elba">Idris Elba</a>) initially relegates her to a safe but boring parking-duty assignment. Meanwhile, the rest of the ZPD is busy investigating 14 missing-mammal cases -- all predators. One day on the job, Judy encounters sly fox Nick Wilde (<a href="https://www.commonsensemedia.org/search/Jason%20Bateman">Jason Bateman</a>), who cheerfully hustles her. But she ends up hustling him right back after promising a worried otter that she'll find her missing husband; with only 48 hours to crack the case if she wants to keep her badge, Judy realizes her best bet is to enlist Nick -- who has plenty of connections -- to help her figure out who's behind the predator kidnappings that are threatening Zootopia's peace.""",
#          'Educational Value':'content-grid-2', 'Positive Messages':'content-grid-5'
#         }
# s.train(url1, data1)

# url2 = f'file:///{path_file2}'
# data2 = s.scrape(url2)
# print(data2)
